<a href="https://colab.research.google.com/github/Moe-phantom/Beyondinfinity/blob/main/kepler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
Importing the necassary libraries

SyntaxError: invalid syntax (ipython-input-3564875100.py, line 1)

In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import joblib

loading the dataset of KEPLER

In [7]:
df = pd.read_csv('/content/cumulative_2025.09.25_07.14.54.csv',
                 comment='#',            # ignore lines that start with #
                 on_bad_lines='skip')    # drop any malformed rows


Downsampling the Data to solve the high imbalance problem

In [32]:
# Separate the classes
df_majority = df[df['koi_disposition']=='FALSE POSITIVE']
df_minority_confirmed = df[df['koi_disposition']=='CONFIRMED']
df_minority_candidate = df[df['koi_disposition']=='CANDIDATE']

# Downsample the majority class
df_majority_downsampled = df_majority.sample(n=3800, random_state=42)

# Combine the downsampled majority class with the minority classes
df = pd.concat([df_majority_downsampled, df_minority_confirmed, df_minority_candidate])

# Display the new class distribution
print(df['koi_disposition'].value_counts())

koi_disposition
FALSE POSITIVE    3800
CONFIRMED         2746
CANDIDATE         1979
Name: count, dtype: int64


In [33]:
df = df[df['koi_disposition'].isin(['CONFIRMED','CANDIDATE','FALSE POSITIVE'])]

In [34]:
label_map = {'CONFIRMED': 2, 'CANDIDATE': 1, 'FALSE POSITIVE': 0}
df['label'] = df['koi_disposition'].map(label_map)

In [35]:
assert df['label'].isna().sum() == 0

In [36]:
df.head()

,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,...,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag,label
3887,5632701,K04213.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.0,0,0,1,1,...,4.534,0.022,-0.093,0.881,0.108,-0.043,295.42047,40.842659,13.997,0
3199,6889235,K00074.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.0,1,0,0,0,...,4.205,0.136,-0.221,1.941,0.795,-0.428,298.32425,42.388470,10.960,0
9526,3533483,K08088.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.0,1,0,0,0,...,4.581,0.025,-0.144,0.813,0.169,-0.068,288.10904,38.652012,14.664,0
9265,10471204,K08018.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.0,0,0,1,1,...,4.538,0.034,-0.184,0.883,0.245,-0.077,292.20523,47.648300,15.824,0
6487,6289897,K05260.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.0,0,0,1,1,...,3.921,0.294,-0.105,1.900,0.406,-0.660,293.37396,41.624859,13.357,0


In [37]:
features = [
    'koi_fpflag_nt','koi_fpflag_ss','koi_fpflag_co','koi_fpflag_ec',
    'koi_period','koi_depth','koi_duration','koi_model_snr',
    'koi_prad','koi_srad','koi_impact','koi_steff','koi_slogg'
]

X = df[features]
y = df['label']

In [38]:
X = X.fillna(X.median())
X.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_depth,koi_duration,koi_model_snr,koi_prad,koi_srad,koi_impact,koi_steff,koi_slogg
3887,0,0,1,1,3.675153,56.0,7.0830,16.7,0.71,0.881,0.730,5968.0,4.534
3199,1,0,0,0,5.188672,548.9,5.0534,331.6,5.20,1.941,0.884,9523.0,4.205
9526,1,0,0,0,308.185801,708.7,3.6890,9.5,2.12,0.813,0.023,5380.0,4.581
9265,0,0,1,1,0.933716,61.3,4.3440,10.3,0.70,0.883,0.401,5734.0,4.538
6487,0,0,1,1,1.559357,49.7,2.8920,17.9,1.39,1.900,0.568,6475.0,3.921


In [39]:
num_rows = df.shape[0]
print(f"The number of rows in the DataFrame is: {num_rows}")

The number of rows in the DataFrame is: 8525


In [40]:
print(df['koi_disposition'].value_counts())

koi_disposition
FALSE POSITIVE    3800
CONFIRMED         2746
CANDIDATE         1979
Name: count, dtype: int64


In [41]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [42]:
clf = lgb.LGBMClassifier(
    n_estimators=500,
    boosting_type='dart',
    learning_rate=0.05,
    num_leaves=31,
    objective='multiclass',
    num_class=3,
    random_state=42
)

In [43]:
clf.fit(X_train, y_train)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001136 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2303
[LightGBM] [Info] Number of data points in the train set: 5967, number of used features: 13
[LightGBM] [Info] Start training from score -0.807918
[LightGBM] [Info] Start training from score -1.460544
[LightGBM] [Info] Start training from score -1.132878


LGBMClassifier(boosting_type='dart', learning_rate=0.05, n_estimators=500,
               num_class=3, objective='multiclass', random_state=42)

In [44]:
y_pred_proba = clf.predict_proba(X_test)
y_pred = clf.predict(X_test)

In [ ]:
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=label_map.keys()))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Classification Report:
                 precision    recall  f1-score   support

     CONFIRMED       0.99      0.99      0.99      1452
     CANDIDATE       0.83      0.83      0.83       594
FALSE POSITIVE       0.89      0.89      0.89       824

      accuracy                           0.93      2870
     macro avg       0.90      0.90      0.90      2870
  weighted avg       0.93      0.93      0.93      2870

Confusion Matrix:
 [[1437   14    1]
 [   9  493   92]
 [   6   84  734]]


In [45]:


# AUC (macro average)
auc_macro = roc_auc_score(y_test, y_pred_proba, multi_class='ovo', average='macro')
print("Macro AUC:", auc_macro)

Macro AUC: 0.9747812149151004


In [46]:
from xgboost import XGBClassifier

xgb = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="mlogloss",
    scale_pos_weight=1  # can tune for imbalance
)


In [47]:

xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
y_proba = xgb.predict_proba(X_test)


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [08:58:51] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "scale_pos_weight" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [48]:

print(classification_report(y_test, y_pred))
print("Macro AUC:", roc_auc_score(y_test, y_proba, multi_class="ovr", average="macro"))


              precision    recall  f1-score   support

           0       0.99      0.98      0.99      1140
           1       0.82      0.80      0.81       594
           2       0.88      0.89      0.89       824

    accuracy                           0.91      2558
   macro avg       0.90      0.89      0.90      2558
weighted avg       0.91      0.91      0.91      2558

Macro AUC: 0.9806824633058855


In [49]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Confusion Matrix:
 [[1122   17    1]
 [  13  478  103]
 [   2   85  737]]


In [50]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Scale only X, not y
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

mlp = MLPClassifier(
    hidden_layer_sizes=(128, 64),
    activation='relu',
    solver='adam',
    max_iter=300,
    random_state=42
)

mlp.fit(X_train_scaled, y_train)
y_pred = mlp.predict(X_test_scaled)
y_proba = mlp.predict_proba(X_test_scaled)

print(classification_report(y_test, y_pred))
print("Macro AUC:", roc_auc_score(y_test, y_proba, multi_class="ovr", average="macro"))


              precision    recall  f1-score   support

           0       0.99      0.98      0.99      1140
           1       0.80      0.79      0.80       594
           2       0.86      0.88      0.87       824

    accuracy                           0.90      2558
   macro avg       0.88      0.88      0.88      2558
weighted avg       0.90      0.90      0.90      2558

Macro AUC: 0.9721494457124353


In [51]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Confusion Matrix:
 [[1119   16    5]
 [   7  471  116]
 [   2  100  722]]


In [52]:
# Save the LightGBM model
joblib.dump(clf, 'lightgbm_model.pkl')
print("LightGBM model saved as lightgbm_model.pkl")

LightGBM model saved as lightgbm_model.pkl


In [53]:
# Save the XGBoost model
joblib.dump(xgb, 'xgboost_model.pkl')
print("XGBoost model saved as xgboost_model.pkl")

XGBoost model saved as xgboost_model.pkl


In [54]:
# Save the MLP model
joblib.dump(mlp, 'mlp_model.pkl')
print("MLP model saved as mlp_model.pkl")

MLP model saved as mlp_model.pkl
